In [1]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
import joblib
from datetime import datetime
import time
import os

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Definir las columnas categóricas, numéricas y ordinales
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 2: Crear el preprocesador de columnas (escalado numérico y OneHotEncoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Escalado para las columnas numéricas
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)  # OneHotEncoding con handle_unknown='ignore'
    ]
)

# Paso 3: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False, cat_features=categorical_cols))
])

# Paso 4: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 5: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 6: Función para evaluar un modelo y medir el tiempo de procesamiento
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    return cv_results, elapsed_time

# Paso 7: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 8: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)
print("\nResultados finales:")
print(df_resultados)

# Paso 9: Seleccionar el mejor modelo en 'accuracy'
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEl mejor modelo es: {mejor_modelo['modelo']} con una accuracy media de: {mejor_modelo['accuracy_mean']}")

# Paso 10: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    os.makedirs('data/modelos_entrenamiento', exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear el nombre del archivo
    nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
    
    # Guardar el modelo
    joblib.dump(modelo, f'data/modelos_entrenamiento/{nombre_archivo}')
    print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el mejor modelo
guardar_modelo_con_nombre(mejor_modelo['modelo_pipeline'], mejor_modelo['modelo'])



Evaluando Random Forest...

Evaluando XGBoost...


c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.


Evaluando CatBoost...

Evaluando Stacking...


c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:06:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\erika\Documents\github\erika_alvares\G1_ML_Airline_Erika\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:06:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.


Resultados finales:
          modelo  accuracy_mean  precision_mean  recall_mean   f1_mean  \
0  Random Forest       0.766685        0.734464     0.723166  0.728758   
1        XGBoost       0.803599        0.768195     0.783237  0.775624   
2       CatBoost       0.960326        0.969199     0.938281  0.953487   
3       Stacking       0.956918        0.961452     0.938214  0.949689   

   roc_auc_mean  tiempo_procesamiento  \
0      0.761570             68.364781   
1      0.801205              2.316897   
2      0.957734             24.514004   
3      0.954720            487.611634   

                                     modelo_pipeline  
0  (ColumnTransformer(transformers=[('num', Stand...  
1  (ColumnTransformer(transformers=[('num', Stand...  
2  (<catboost.core.CatBoostClassifier object at 0...  
3  StackingClassifier(cv=5,\n                   e...  

El mejor modelo es: CatBoost con una accuracy media de: 0.960325907153611
Modelo guardado exitosamente con el nombre: CatBoost